In [1]:
import pandas as pd
from haversine import haversine

In [2]:
city_list=["광주", "울산", "전주", "창원"]
i=3
fire_incident_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+" 화재 정보.csv")
accident_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+"_교통사고_통계.csv")
school_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+" 초중고.csv")
bus_station_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+"시_버스 정류장.csv")
gas_station_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+"_충전소_주유소.csv")
weight_df=pd.DataFrame([[0]*7]*len(gas_station_df), columns=['주유소명', '버정거리총합', '상호개수', '교통사고건수','보호구역개수','화재발생건수','총 가중치'])
weight_df['주유소명']=gas_station_df['상호'][:]
## 추가추가

In [3]:
fire_incident_df.head(3)

,조사서번호,시도,시군구,읍면동,주소,위도,경도
0,'170103010226706,경상남도,창원시 마산합포구,월영동,경상남도 창원시 마산합포구 월영동,35.179212,128.55769
1,'170101204245220,경상남도,창원시 마산합포구,해운동,경상남도 창원시 마산합포구 해운동,35.179212,128.55769
2,'170102165952168,경상남도,창원시 의창구,대산면,경상남도 창원시 의창구 대산면,35.179212,128.55769


In [4]:
accident_df.head(3)

,시군구,도로명,주소,사고건수,사망자수,중상자수,경상자수,부상신고자수,위도,경도
0,경남 창원시,원이대로579번길,경남 창원시 원이대로579번길,1,0,1,0,0,35.229844,128.680001
1,경남 창원시,중앙대로161번길,경남 창원시 중앙대로161번길,1,0,0,1,0,35.460600,128.213200
2,경남 창원시,외동반림로248번길,경남 창원시 외동반림로248번길,1,0,1,0,0,35.237904,128.681376


In [5]:
bus_station_df.head(3)

,관리번호,행정구역,정류장명,주소,경도,위도,Unnamed: 6
0,100201,동읍,합산,NaN,128.677387,35.337581,NaN
1,100202,동읍,합산,NaN,128.677291,35.337657,NaN
2,100203,동읍,용산입구,NaN,128.678880,35.332786,NaN


In [6]:
gas_station_df.head(3)

,지역,상호,타입,주소,위도,경도
0,경상남도,SK행복충전 불모가스충전소,충전소,경남 창원시 성산구 창원대로 1310 (불모산동),35.191633,128.736469
1,경상남도,동부2,충전소,경남 창원시 진해구 진해대로 732 (석동),35.162215,128.701874
2,경상남도,(유)공단가스산업,충전소,경남 창원시 마산회원구 봉암공단8길 2,35.214543,128.599861


## 가산

가능 : 타 주유소, 교통량(교통 사고), 버스 정류장

working on : 차고지 위치

In [7]:
# 거리 총합 리스트
def 거리총합리스트(data1, data2, name_col):
    # distance_list=[]

    for i in range(len(data1)):
        total=0
        for j in range(len(data2)):
            total+=haversine((data1.loc[i]['위도'],data1.loc[i]['경도']),(data2.loc[j]['위도'],data2.loc[j]['경도']), unit = 'km')
        # distance_list.append((data1.loc[i][name_col],total))
        weight_df['버정거리총합'][i]=total
    

In [8]:
# p 반경 이내 개수 리스트(타 주유소)
def 반경이내_개수_리스트(p, data1, data2, name_col):
    count_list=[]
    for i in range(len(data1)):
        count=0
        for j in range(len(data2)):
            dist=haversine((data1.loc[i]['위도'],data1.loc[i]['경도']),(data2.loc[j]['위도'],data2.loc[j]['경도']), unit = 'km')
            if dist<=p:
                # 교통사고 데이터 사고 건수로 count!
                if ('사고건수' in data2.columns):
                    count+=data2.loc[j,'사고건수']
                else:
                    count+=1
        #count_list.append((data1.loc[i][name_col],count-1)) 상호명 매핑 화긴~
        if (count == 0):
            count_list.append(count)
        else:
            count_list.append(count-1)# 자기 자신 제외

    return count_list
   

In [9]:
""" 이건 km
거리총합리스트(gas_station_df, bus_station_df, '상호')
weight_df['버정거리총합']=max(weight_df['버정거리총합'])-weight_df['버정거리총합']
weight_df
""" # 이건 개수
버정거리총합 = 반경이내_개수_리스트(2, gas_station_df, bus_station_df,'상호')
weight_df['버정거리총합']=버정거리총합
weight_df

,주유소명,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,총 가중치
0,SK행복충전 불모가스충전소,7,0,0,0,0,0
1,동부2,78,0,0,0,0,0
2,(유)공단가스산업,68,0,0,0,0,0
3,광신직영(두대충전소),86,0,0,0,0,0
4,SK행복충전 유남충전소(주)배성에너지,90,0,0,0,0,0
...,...,...,...,...,...,...,...
216,판도라주유소,32,0,0,0,0,0
217,진웅,48,0,0,0,0,0
218,광신직영(호원셀프),65,0,0,0,0,0
219,야구장주유소,152,0,0,0,0,0


In [10]:
상호개수 = 반경이내_개수_리스트(2, gas_station_df, gas_station_df,'상호')
weight_df['상호개수']=상호개수
weight_df

,주유소명,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,총 가중치
0,SK행복충전 불모가스충전소,7,3,0,0,0,0
1,동부2,78,10,0,0,0,0
2,(유)공단가스산업,68,18,0,0,0,0
3,광신직영(두대충전소),86,14,0,0,0,0
4,SK행복충전 유남충전소(주)배성에너지,90,15,0,0,0,0
...,...,...,...,...,...,...,...
216,판도라주유소,32,4,0,0,0,0
217,진웅,48,5,0,0,0,0
218,광신직영(호원셀프),65,10,0,0,0,0
219,야구장주유소,152,30,0,0,0,0


In [11]:
교통사고건수=반경이내_개수_리스트(2, gas_station_df, accident_df,'상호')
weight_df['교통사고건수']=교통사고건수
weight_df

,주유소명,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,총 가중치
0,SK행복충전 불모가스충전소,7,3,0,0,0,0
1,동부2,78,10,172,0,0,0
2,(유)공단가스산업,68,18,127,0,0,0
3,광신직영(두대충전소),86,14,177,0,0,0
4,SK행복충전 유남충전소(주)배성에너지,90,15,203,0,0,0
...,...,...,...,...,...,...,...
216,판도라주유소,32,4,20,0,0,0
217,진웅,48,5,150,0,0,0
218,광신직영(호원셀프),65,10,33,0,0,0
219,야구장주유소,152,30,308,0,0,0


## 감점

가능 : 어린이집, 유초중고(2km)

working on : 화재

In [12]:
fire_incident_df.head(3)

,조사서번호,시도,시군구,읍면동,주소,위도,경도
0,'170103010226706,경상남도,창원시 마산합포구,월영동,경상남도 창원시 마산합포구 월영동,35.179212,128.55769
1,'170101204245220,경상남도,창원시 마산합포구,해운동,경상남도 창원시 마산합포구 해운동,35.179212,128.55769
2,'170102165952168,경상남도,창원시 의창구,대산면,경상남도 창원시 의창구 대산면,35.179212,128.55769


In [13]:
gas_station_df.head(3)

,지역,상호,타입,주소,위도,경도
0,경상남도,SK행복충전 불모가스충전소,충전소,경남 창원시 성산구 창원대로 1310 (불모산동),35.191633,128.736469
1,경상남도,동부2,충전소,경남 창원시 진해구 진해대로 732 (석동),35.162215,128.701874
2,경상남도,(유)공단가스산업,충전소,경남 창원시 마산회원구 봉암공단8길 2,35.214543,128.599861


In [14]:
school_df.head(3)

,학교명,주소,위도,경도
0,경상고등학교,경상남도 창원시 의창구 금강로379번길 46. 경상고등학교 (소계동.경상고등학교),35.260853,128.599645
1,마산가포고등학교,경상남도 창원시 마산합포구 가포로 262(가포동. 마산가포고등학교),35.167301,128.571367
2,마산고등학교,경상남도 창원시 마산합포구 심온길 33(완월동. 마산고등학교),35.200348,128.561745


In [15]:
#kinder_df.head(3)

In [16]:
#유치원,,
#보호구역개수=반경이내_개수_리스트(2, gas_station_df, kinder_df, '상호')
#weight_df['보호구역개수']=보호구역개수

In [17]:
보호구역개수=반경이내_개수_리스트(2, gas_station_df, school_df, '상호')
weight_df['보호구역개수']=보호구역개수
weight_df

,주유소명,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,총 가중치
0,SK행복충전 불모가스충전소,7,3,0,0,0,0
1,동부2,78,10,172,25,0,0
2,(유)공단가스산업,68,18,127,11,0,0
3,광신직영(두대충전소),86,14,177,17,0,0
4,SK행복충전 유남충전소(주)배성에너지,90,15,203,17,0,0
...,...,...,...,...,...,...,...
216,판도라주유소,32,4,20,4,0,0
217,진웅,48,5,150,3,0,0
218,광신직영(호원셀프),65,10,33,5,0,0
219,야구장주유소,152,30,308,45,0,0


In [18]:
화재발생건수=반경이내_개수_리스트(2, gas_station_df, fire_incident_df, '상호')
weight_df['화재발생건수']=화재발생건수
weight_df

,주유소명,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,총 가중치
0,SK행복충전 불모가스충전소,7,3,0,0,1,0
1,동부2,78,10,172,25,26,0
2,(유)공단가스산업,68,18,127,11,17,0
3,광신직영(두대충전소),86,14,177,17,30,0
4,SK행복충전 유남충전소(주)배성에너지,90,15,203,17,43,0
...,...,...,...,...,...,...,...
216,판도라주유소,32,4,20,4,16,0
217,진웅,48,5,150,3,6,0
218,광신직영(호원셀프),65,10,33,5,30,0
219,야구장주유소,152,30,308,45,51,0


## 스케일링

In [19]:
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler()
print(minMaxScaler.fit(weight_df.iloc[:,[1,2,3,4,5]]))
weight_df_stdScaled = minMaxScaler.transform(weight_df.iloc[:,[1,2,3,4,5]])
weight_df_stdScaled=pd.DataFrame(weight_df_stdScaled, columns=["버정거리총합","상호개수","교통사고건수","보호구역개수","화재발생건수"])
weight_df_stdScaled["주유소명"]=weight_df["주유소명"][:]
weight_df_stdScaled

MinMaxScaler()


,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,주유소명
0,0.006667,0.09375,0.000000,0.000000,0.013699,SK행복충전 불모가스충전소
1,0.480000,0.31250,0.320896,0.431034,0.356164,동부2
2,0.413333,0.56250,0.236940,0.189655,0.232877,(유)공단가스산업
3,0.533333,0.43750,0.330224,0.293103,0.410959,광신직영(두대충전소)
4,0.560000,0.46875,0.378731,0.293103,0.589041,SK행복충전 유남충전소(주)배성에너지
...,...,...,...,...,...,...
216,0.173333,0.12500,0.037313,0.068966,0.219178,판도라주유소
217,0.280000,0.15625,0.279851,0.051724,0.082192,진웅
218,0.393333,0.31250,0.061567,0.086207,0.410959,광신직영(호원셀프)
219,0.973333,0.93750,0.574627,0.775862,0.698630,야구장주유소


In [20]:
weight_df_stdScaled.to_csv(city_list[i]+" 스케일링완료.csv")

# ########## 여기까지1 - 스케일링 완, 가중치 미완(lr용) ############

## 가중치 매핑 함수
새로운 데이터 프레임 만들어서 '상호'와 가중치1,2,3,4,5 넣고 총합 마지막에 하고 sort해서 [:20] 출력

### 가산
주유소_count_list=반경이내_개수_리스트(2, gas_station_df, gas_station_df,'상호')

distance_list=거리총합리스트(gas_station_df, bus_station_df, '상호')

교통사고_count_list=반경이내_개수_리스트(2, gas_station_df, accident_df,'상호')
### 감점
유치원_count_list=반경이내_개수_리스트(2, gas_station_df, kinder_df, '상호')

유초중고_count_list=반경이내_개수_리스트(2, gas_station_df, school_df, '상호')

화재_count_list=반경이내_개수_리스트(2, gas_station_df, fire_incident_df, '상호')

In [21]:
# **주유소 순서 헤치지 말 것**
#columns=['버정거리총합', '상호개수', '교통사고건수','보호구역개수','화재발생건수',주유소명'])
def 가중치추가(df, ratio_list):
    for i in range(0,len(df.loc[0])-2):
        df['총 가중치']+=df.iloc[:,i]*ratio_list[i]# weight_df는 주유소명부터, ratio_list는 버정거리총합부터 시작하므로
    


In [22]:
weight_df_stdScaled["총 가중치"]=0 # 혹시 모를 사고에 대비
ratio_list=[0.001,0.1,0.1,-0.1,-0.1] # 뒤에 두개 음수 필수
가중치추가(weight_df_stdScaled, ratio_list)
weight_df_stdScaled.head(5)

,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,주유소명,총 가중치
0,0.006667,0.09375,0.000000,0.000000,0.013699,SK행복충전 불모가스충전소,0.008012
1,0.480000,0.31250,0.320896,0.431034,0.356164,동부2,-0.014900
2,0.413333,0.56250,0.236940,0.189655,0.232877,(유)공단가스산업,0.038104
3,0.533333,0.43750,0.330224,0.293103,0.410959,광신직영(두대충전소),0.006899
4,0.560000,0.46875,0.378731,0.293103,0.589041,SK행복충전 유남충전소(주)배성에너지,-0.002906


In [48]:
weight_df_stdScaled.sort_values(by=["총 가중치", "주유소명"], ascending=[False, False], inplace=True) 
weight_df_stdScaled = weight_df_stdScaled.reset_index(drop=True)
weight_df_stdScaled

,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,주유소명,총 가중치
0,0.733333,0.71875,0.809701,0.431034,0.369863,CJ대한통운㈜ 오동동셀프주유소,0.073489
1,0.893333,0.90625,1.000000,0.689655,0.506849,금강주유소,0.071868
2,0.700000,0.65625,0.800373,0.431034,0.369863,지에스칼텍스㈜ 마산월드주유소,0.066273
3,0.853333,0.87500,0.930970,0.758621,0.506849,성운주유소,0.054903
4,0.880000,0.90625,0.936567,0.706897,0.602740,합포주유소,0.054198
...,...,...,...,...,...,...,...
216,0.793333,0.18750,0.598881,0.844828,0.767123,신월주유소,-0.081764
217,0.666667,0.15625,0.335821,0.793103,0.534247,지에스칼텍스㈜ 감동주유소,-0.082861
218,0.706667,0.15625,0.333955,0.810345,0.534247,남창원농협주유소,-0.084732
219,0.733333,0.21875,0.647388,0.931034,0.890411,동부3,-0.094797


In [24]:
#weight_df_stdScaled_sorted.to_csv(city_list[2]+" 가중치.csv")

# ########## 여기까지2 - 스케일링, 가중치 모두 완 (f1용)

## 정확도 나타내기
- lr
- f1

In [49]:
# 실제 data
guess_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+"_충전소.csv")
guess_df

IndexError: list index out of range

In [50]:
# 실제를 전체 주유소에 대입
labels_df=pd.DataFrame([[0]*2]*len(weight_df_stdScaled), columns=['주유소명','label'])
labels_df['주유소명']=weight_df_stdScaled['주유소명']
for i in range(len(guess_df)):
    if labels_df['주유소명'][i]==guess_df['충전소'][i]:
        labels_df['label'][i]+=1
labels_df

,주유소명,label
0,CJ대한통운㈜ 오동동셀프주유소,0
1,금강주유소,0
2,지에스칼텍스㈜ 마산월드주유소,0
3,성운주유소,0
4,합포주유소,0
...,...,...
216,신월주유소,0
217,지에스칼텍스㈜ 감동주유소,0
218,남창원농협주유소,0
219,동부3,0


In [51]:
weight_df_stdScaled_sorted

In [52]:
# weight_df_sorted 상위 10개 label에 1 넣음
eval_df=pd.DataFrame([[0]*2]*len(weight_df_stdScaled), columns=['주유소명','label'])
weight_df_stdScaled.sort_values(by=["총 가중치", "주유소명"], ascending=[False, False], inplace=True) 
eval_df['주유소명']=weight_df_stdScaled['주유소명'][:]
eval_df['label'][:len(guess_df)]=1#일단 그냥 추측 개수 같게 했다,,
eval_df

<ipython-input-52-fa74eba53aec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df['label'][:len(guess_df)]=1#일단 그냥 추측 개수 같게 했다,,
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,주유소명,label
0,CJ대한통운㈜ 오동동셀프주유소,1
1,금강주유소,1
2,지에스칼텍스㈜ 마산월드주유소,1
3,성운주유소,1
4,합포주유소,1
...,...,...
216,신월주유소,0
217,지에스칼텍스㈜ 감동주유소,0
218,남창원농협주유소,0
219,동부3,0


In [53]:
# 전체 충전소 개수 중 guess의 개수 안에 있는 충전소 개수
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
eval_df['label']=0
# eval_df.columns=['주유소명','label']
for i in range(len(guess_df)):
    if guess_df['충전소'][i]==eval_df['주유소명'][i]:
        eval_df['label'][i]+=1

eval_df.sort_values(by=["주유소명"], ascending=[False]) 
guess_df.sort_values(by=["충전소"], ascending=[False]) 
# 원래 친구
labels=labels_df['label']
# 추정한 친구
guesses=eval_df['label']
guesses

0      0
1      0
2      0
3      0
4      0
      ..
216    0
217    0
218    0
219    0
220    0
Name: label, Length: 221, dtype: int64

In [54]:
print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses))
print(precision_score(labels, guesses))
print(f1_score(labels, guesses))

1.0
0.0
0.0
0.0


C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


## 동 매핑

동이랑 통행속도 매핑

gas_station_df['주소']-도로 이름 나오니까 그냥 사용

travel_speed['도로명'], ['속도']

In [30]:
# 그냥 대충 만든 거
travel_speed=pd.DataFrame([['혁신로', 20],['기지로', 30],['쑥고개로', 10],['권삼득로', 33]], columns=['도로명','속도'])
travel_speed

,도로명,속도
0,혁신로,20
1,기지로,30
2,쑥고개로,10
3,권삼득로,33


In [31]:
# 가중치 데이터 프레임
weight_df=pd.DataFrame([[0]*5]*len(gas_station_df), columns=['주유소_이름', '주소','통행속도_w', 'a', 'b'])
weight_df['주유소_이름']=gas_station_df['상호'][:]
weight_df['주소']=gas_station_df['주소'][:]
weight_df

,주유소_이름,주소,통행속도_w,a,b
0,SK행복충전 불모가스충전소,경남 창원시 성산구 창원대로 1310 (불모산동),0,0,0
1,동부2,경남 창원시 진해구 진해대로 732 (석동),0,0,0
2,(유)공단가스산업,경남 창원시 마산회원구 봉암공단8길 2,0,0,0
3,광신직영(두대충전소),경남 창원시 의창구 남면로 3 (대원동),0,0,0
4,SK행복충전 유남충전소(주)배성에너지,경남 창원시 의창구 창원대로 314 (팔용동),0,0,0
...,...,...,...,...,...
216,판도라주유소,경남 창원시 마산합포구 진동면 삼진의거대로 753,0,0,0
217,진웅,경남 창원시 진해구 웅천로 156 (북부동),0,0,0
218,광신직영(호원셀프),경남 창원시 마산회원구 내서읍 호원로 141,0,0,0
219,야구장주유소,경남 창원시 마산합포구 용마로 62,0,0,0


In [32]:
for i in range(len(travel_speed)):
    for j in range(len(gas_station_df)):
        if travel_speed.loc[i]['도로명'] in gas_station_df.loc[j]['주소']:
            # gas_station_df.loc[j]['상호']
            # 하는 중
            # weight_df.iloc['주유소_이름']['통행속도_w']+=1
weight_df

IndentationError: expected an indented block (<ipython-input-32-3dfeeb1f015a>, line 7)